In [1]:
import json
import requests
import yaml
import pandas as pd
import warnings
from bs4 import BeautifulSoup
import re
import numpy as np
from shapely.geometry import Point
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# key_dict = {'key' : ''}

In [3]:
# dump the key into yaml 
with open('credentials.yml','w') as output:
    yaml.dump(key_dict, output)
    
# get the key from .yaml file
key_dict = yaml.load(open('credentials.yml'), Loader=yaml.FullLoader)  

In [4]:
key_dict = yaml.load(open("credentials.yml"), Loader=yaml.FullLoader)
gmaps_key = key_dict["key"]
api_key = gmaps_key

### Scraping Coop locations in Zürich

In [5]:
page_Aldi = requests.get("https://www.profital.ch/de/flr/72133-aldi-suisse-filialen/2699")
print(page_Aldi.status_code) ## prints 200 as status code
#print(page.content[:3000]) ## prints the entire webpage content
soup_Aldi = BeautifulSoup(page_Aldi.content, "html.parser")
type(soup_Aldi)

200


bs4.BeautifulSoup

In [6]:
soup_Aldi.findAll('address');

In [7]:
Loc_Aldi=[]
Loc_Aldi.append(soup_Aldi.findAll('address'))
page_num_Aldi=len(Loc_Aldi)
page_num_Aldi

1

In [9]:
Address_Aldi=[]
Zip_Aldi=[]
for i in range(0,page_num_Aldi):
    for j in range(0,len(Loc_Aldi[i])):
        Address_Aldi.append(Loc_Aldi[i][j].contents[0].strip())
        Zip_Aldi.append(Loc_Aldi[i][j].contents[2].strip().split()[0])

In [14]:
df_Aldi_Locator = pd.DataFrame(list(zip(Address_Aldi,Zip_Aldi)), columns=["Address", "Zip_Code"])

In [15]:
df_Aldi_Address = pd.DataFrame()
# Combine Address and Zip Code into a single column
df_Aldi_Address["address"] = df_Aldi_Locator["Address"] + ', ' + df_Aldi_Locator["Zip_Code"]
df_Aldi_Address

,address
0,"Stadelhoferstrasse 10, 8001"
1,"Mythenquai 353, 8038"
2,"Albisstrasse 81, 8038"
3,"Witikonerstrasse 279, 8053"
4,"Sihlstrasse 3, 8001"
5,"Zollstrasse 58, 8005"
6,"Badenerstrasse 565, 8048"
7,"Hohlstrasse 473, 8048"
8,"Baumackerstrasse 35, 8050"
9,"Überlandstrasse 441, 8051"


### Getting store locations from Google Maps and mapping the location to Kreis name

In [16]:
from shapely.geometry import shape, GeometryCollection, Point

In [17]:
# Function to get latitude and longitude from Google Maps API
def get_lat_lng(api_key, query):
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
    req = requests.get(url + "query=" + query + "&key=" + api_key)
    places_json = req.json()
    my_result = places_json.get("results", [])

    if my_result:
        location = my_result[0].get("geometry", {}).get("location", {})
        lat, lng = location.get("lat"), location.get("lng")
        return lat, lng
    else:
        return None, None

In [18]:
# Function to get kreis_name from GeoJSON data
def get_kreis_name(point, geojson_data):
    for feature in geojson_data["features"]:
        polygon = shape(feature["geometry"])
        if polygon.contains(point):
            return feature["properties"]["bezeichnung"]
    return None

In [20]:
# Load GeoJSON data
with open('data/stzh.adm_stadtkreise_a.json', 'r') as f:
    geojson_data = json.load(f)

# Create a DataFrame to store the results
columns = ["address", "latitude", "longitude", "kreis_name"]
aldi_df = pd.DataFrame(columns=columns)

# Iterate through addresses in the DataFrame
for index, row in df_Aldi_Address.iterrows():
    address = row["address"]
    
    # Get latitude and longitude from Google Maps API
    lat, lng = get_lat_lng(api_key, address)
    
    # If coordinates are found, create a Point object
    if lat is not None and lng is not None:
        point = Point(lng, lat)
        
        # Get kreis_name from GeoJSON data
        kreis_name = get_kreis_name(point, geojson_data)
        
        # Append results to DataFrame
        aldi_df = pd.concat([aldi_df, pd.DataFrame([{"address": address, "latitude": lat, "longitude": lng, "kreis_name": kreis_name}])], ignore_index=True)

# Display the final DataFrame
aldi_df

,address,latitude,longitude,kreis_name
0,"Stadelhoferstrasse 10, 8001",47.366789,8.548001,Kreis 1
1,"Mythenquai 353, 8038",47.345470,8.534889,Kreis 2
2,"Albisstrasse 81, 8038",47.340868,8.530663,Kreis 2
3,"Witikonerstrasse 279, 8053",47.358731,8.585599,Kreis 7
4,"Sihlstrasse 3, 8001",47.373727,8.537653,Kreis 1
5,"Zollstrasse 58, 8005",47.380852,8.533312,Kreis 5
6,"Badenerstrasse 565, 8048",47.383793,8.497718,Kreis 9
7,"Hohlstrasse 473, 8048",47.388231,8.496790,Kreis 9
8,"Baumackerstrasse 35, 8050",47.408720,8.543906,Kreis 11
9,"Überlandstrasse 441, 8051",47.405963,8.591885,Kreis 12


In [21]:
aldi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   address     12 non-null     object 
 1   latitude    12 non-null     float64
 2   longitude   12 non-null     float64
 3   kreis_name  12 non-null     object 
dtypes: float64(2), object(2)
memory usage: 516.0+ bytes


In [22]:
# Save DataFrame to CSV
aldi_df.to_csv('aldi_data.csv', index=False)